## Reparametrize force fields
Automatized refining of a dihedral angle to fit to QM data. GAFF2 is known to not have well parametrized sp2-sp2 single bond dihedrals. In this example we will reparametrize the rotatable bond in acrolein.

We start by importing VeloxChem and defining our molecule object.

In [ ]:
import veloxchem as vlx

In [ ]:
molecule = vlx.Molecule.read_smiles('C=CC=O')
molecule.show(atom_indices=True)

Now we generate a force field and write force field files

In [ ]:
ff_gen = vlx.MMForceFieldGenerator()
ff_gen.create_topology(molecule)
ff_gen.write_openmm_files('acrolein_initial', 'MOL')

We can then identify which bond is rotatable.

In [ ]:
rot_bonds = ff_gen.rotatable_bonds
print(rot_bonds)

Now we will make the first round of reparametrization. The ```reparametrize_dihedrals()``` function will perform a scan of the dihedral at the HF/6-31G* level (can be changed) and it will refit the dihedral force constants associated with the rotatable bond. The QM results will be stored in an xyz-file with the name of the dihedral scanned.

In [ ]:
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.xcfun = 'BLYP'
scf_drv.ri_coulomb = True
scf_drv.grid_level = 2
scf_drv.conv_thresh = 1e-3
basis = vlx.MolecularBasis.read(molecule, 'def2-sv(p)')
scan_results = ff_gen.scan_dihedral(scf_drv, basis, [2,3], scan_range=[180, 360], n_points=7)
ff_gen.reparameterize_dihedrals([2,3], scan_results, visualize=True)

In [ ]:
ff_gen.add_dihedral([1,2,3,4], 2, 0, 1)
ff_gen.reparameterize_dihedrals([2,3], scan_results, visualize=True)

In [ ]:
ff_gen.write_openmm_files('acrolein_final', 'MOL')